In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import requests
import datetime

#from google.cloud import bigquery
#client = bigquery.Client()

In [3]:
lookback = 7

today = datetime.datetime.today()
start_date = (today - datetime.timedelta(days=lookback)).strftime('%Y-%m-%d')
end_date = today.strftime('%Y-%m-%d')

Accounts created daily

In [4]:
url = f'https://data.ripple.com/v2/stats/?start={start_date}&end={end_date}&interval=day&family=metric&metrics=accounts_created'
res = requests.get(url)
xrp_accts = pd.DataFrame(res.json()['stats'])
xrp_accts.tail()

,accounts_created,date
3,495,2019-12-18T00:00:00Z
4,436,2019-12-19T00:00:00Z
5,465,2019-12-20T00:00:00Z
6,410,2019-12-21T00:00:00Z
7,109,2019-12-22T00:00:00Z


Network stats: number of nodes and validators

In [5]:
date_list = [x.strftime('%Y-%m-%dT%H:%M:%SZ') for x in list(pd.date_range((today - datetime.timedelta(days=lookback)).strftime('%Y%m%d'),today.strftime('%Y%m%d'),freq='1D'))]
node_list = []

for d in date_list:
    url = f'https://data.ripple.com/v2/network/topology?verbose=true&date={d}'
    res = requests.get(url)
    try:
        node_list.append(res.json()['node_count'])
    except:
        node_list.append(None)
        print(f'Error with {d}')
nodes_df = pd.DataFrame({'date': date_list, 'nodes': node_list})
nodes_df

,date,nodes
0,2019-12-15T00:00:00Z,971
1,2019-12-16T00:00:00Z,971
2,2019-12-17T00:00:00Z,977
3,2019-12-18T00:00:00Z,975
4,2019-12-19T00:00:00Z,991
5,2019-12-20T00:00:00Z,1004
6,2019-12-21T00:00:00Z,987
7,2019-12-22T00:00:00Z,980


In [6]:
res = requests.get(f'https://data.ripple.com/v2/network/validators')
res.json()['count']

153

Ledger data: transaction count and value

Requires a GBQ account. You can provide service account JSON credentials as an argument. If you want to run as your google user from your PC, you should first install the [Google Cloud SDK](https://cloud.google.com/sdk/), then run:
`gcloud auth application-default login`

In [7]:
def gbq_query(query, query_params=None):
    """
    Run a query against Google Big Query, returning a pandas dataframe of the result.

    Parameters
    ----------
    query: str
        The query string
    query_params: list, optional
        The query parameters to pass into the query string
    """
    client = bigquery.Client()
    job_config = bigquery.QueryJobConfig()
    job_config.query_parameters = query_params
    return client.query(query, job_config=job_config).to_dataframe()

In [12]:
query = """
select
    date(l.CloseTime) as `date`
    , t.TransactionType
    , count(1) as txn_count
    , sum(t.AmountXRP) / 1e6 as txn_value
from `xrpledgerdata.fullhistory.transactions` t
join `xrpledgerdata.fullhistory.ledgers` l
    on t.LedgerIndex = l.LedgerIndex
where t.TransactionResult = "tesSUCCESS"
    and date(l.CloseTime) >= CAST(@start_date AS DATE)
group by 1,2
order by 1 desc, 2
"""

query_params = [
    bigquery.ScalarQueryParameter("start_date", "STRING", start_date)
]

xrp = gbq_query(query,query_params)
xrp

/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnin

,date,TransactionType,txn_count,txn_value
0,2019-10-17,AccountSet,408,NaN
1,2019-10-17,OfferCancel,16136,NaN
2,2019-10-17,OfferCreate,388463,NaN
3,2019-10-17,Payment,31410,3.188668e+08
4,2019-10-17,SetRegularKey,2,NaN
5,2019-10-17,SignerListSet,3,NaN
6,2019-10-17,TrustSet,114,NaN
7,2019-10-16,AccountSet,802,NaN
8,2019-10-16,EscrowCreate,9,3.901000e+03
9,2019-10-16,EscrowFinish,2,NaN
